In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Dropout

In [ ]:
path= '/content/drive/MyDrive/Research/IBM Advanced Data Science/'

In [ ]:
model = load_model(path + 'predicting_location.h5')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 4)                 20        
                                                                 
 dense_10 (Dense)            (None, 128)               640       
                                                                 
 dense_11 (Dense)            (None, 32)                4128      
                                                                 
 dense_12 (Dense)            (None, 6)                 198       
                                                                 
Total params: 4,986
Trainable params: 4,986
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import glob2 as glob
import re
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
pd.option_context("display.max_columns", None)

In [ ]:
# List of files
mainPath = "/content/drive/MyDrive/Research"

files = glob.glob(mainPath + '/beacons_home/**/*.csv', recursive = True)

# Ground truth: training dataset
files_train = [file for file in files if 'Tests' in file]

# Files for testing
files_test = list(set(files) - set(files_train))

In [ ]:
# Check dataframe structure
def extract_info_file(filename, cont):
  lastPart = filename.split(' ')[-1]
  lastPart = lastPart.replace(".csv", "")
  area = lastPart.split('_')[0]
  df = pd.read_csv(filename)
  df['user_at'] = area
  df['file_id'] = cont
  return(df)

In [ ]:
testDF_or = pd.DataFrame()
cont = 1

for file in files_test:
  tmpDF = extract_info_file(file, cont)
  testDF_or = testDF_or.append(tmpDF, ignore_index=True)
  cont += 1

In [ ]:
fixing = {'80:EC:CA:CD:2F:84': 'main',
          '80:EC:CA:CD:2F:42': 'kitchen',
          'D9:C7:16:15:A3:0F': 'living',
          'C2:65:7C:E2:D6:6B': 'study'}

testDF_or['beacon_at'] = testDF_or['MAC Address'].replace(fixing)
testDF_or.head()

,Sr No,Timestamp,Device Name,MAC Address,RSSI,Raw Data,Battery,Ambient Temperature,user_at,file_id,beacon_at
0,1,17-08-2021 17:41:06:782,BMPkitchen,80:EC:CA:CD:2F:42,-80,1a4c0002150112233445566778899aabbccddeeff02716...,-,-,p09,1,kitchen
1,2,17-08-2021 17:41:06:802,-,08:C0:60:8C:D6:B4,-68,1e0600010920021bb6f3cf172774cab879529050eddcfa...,-,-,p09,1,08:C0:60:8C:D6:B4
2,3,17-08-2021 17:41:06:839,BMPmainACD,80:EC:CA:CD:2F:84,-70,1a4c0002150112233445566778899aabbccddeeff02716...,-,-,p09,1,main
3,4,17-08-2021 17:41:06:849,BrtBeacon402,C2:65:7C:E2:D6:6B,-65,-,21,170.1 Deg C,p09,1,study
4,5,17-08-2021 17:41:06:864,BrtBeacon402,D9:C7:16:15:A3:0F,-84,-,21,170.1 Deg C,p09,1,living


In [ ]:
testDF_or.shape

(37266, 11)

In [ ]:
# beacons placed
macAddresses = ['80:EC:CA:CD:2F:42', '80:EC:CA:CD:2F:84', 'C2:65:7C:E2:D6:6B', 'D9:C7:16:15:A3:0F']

In [ ]:
testDF = testDF_or[testDF_or['MAC Address'].isin(macAddresses)]
testDF = testDF.reset_index(drop=True)
testDF.head()

,Sr No,Timestamp,Device Name,MAC Address,RSSI,Raw Data,Battery,Ambient Temperature,user_at,file_id,beacon_at
0,1,17-08-2021 17:41:06:782,BMPkitchen,80:EC:CA:CD:2F:42,-80,1a4c0002150112233445566778899aabbccddeeff02716...,-,-,p09,1,kitchen
1,3,17-08-2021 17:41:06:839,BMPmainACD,80:EC:CA:CD:2F:84,-70,1a4c0002150112233445566778899aabbccddeeff02716...,-,-,p09,1,main
2,4,17-08-2021 17:41:06:849,BrtBeacon402,C2:65:7C:E2:D6:6B,-65,-,21,170.1 Deg C,p09,1,study
3,5,17-08-2021 17:41:06:864,BrtBeacon402,D9:C7:16:15:A3:0F,-84,-,21,170.1 Deg C,p09,1,living
4,10,17-08-2021 17:41:06:947,BMPmainACD,80:EC:CA:CD:2F:84,-71,1a4c0002150112233445566778899aabbccddeeff02716...,-,-,p09,1,main


In [ ]:
testDF.shape

(19720, 11)